In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-4iy3jztf
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-4iy3jztf
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369408 sha256=6242244616523ef01b6c465c1d99936e05921c214a3f1c3472b51548f2df22e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-4vucnrly/wheels/ab/4f/3a/5e51521b55997aa6f0690e095c08824219753128ce8d9969a3
Successfully built clip


In [3]:
import os
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

classes = ["Basketball", "CarRacing", "Fighting", "TableTennis", "Volleyball"]
prompts = ["a photo of a", "a offline sports scene of a", "a online sports scene of a", "a sports event of a", "a sports game of a"]
path = [['jordanvsbirdgen','incredibasketball','nbaelite11'],
        ['ridgeracer','asphalt5', 'gtsport'],
        ['finalblow', 'ufc1', 'realboxing'],
        ['pingpongkingarcade', 'virtualtabletennis', 'worldttchamps'],
        ['summerheatbeachvolley', 'sportschampsbeach', 'volleyballunbound']
        ]
directory = '/content/drive/MyDrive/colab/NewData'

scores = []

for j, p in enumerate(prompts) :
  #text encoding
  text = torch.cat([clip.tokenize(p + " " + c) for c in classes]).to(device)
  print(text)
  with torch.no_grad():
      text_features = model.encode_text(text)
  text_features /= text_features.norm(dim=-1, keepdim=True)

  scores.append([])

  #image encoding and prediction
  for i in range(len(classes)) :
    scores[j].append([])
    for game in path[i] :
      total = 0
      correct = 0

      for filename in os.listdir(directory + "/" + classes[i] + "/" + game):
        f = os.path.join(directory, classes[i])
        f = os.path.join(f, game)
        f = os.path.join(f, filename)

        if os.path.isfile(f):
          image = preprocess(Image.open(f)).unsqueeze(0).to(device)
          #print("isfile")
        else :
          continue

        with torch.no_grad():
          image_features = model.encode_image(image)

        image_features /= image_features.norm(dim=-1, keepdim=True)

        similarity = (image_features @ text_features.T).softmax(dim=-1)
        prediction = torch.argmax(similarity[0])
        total += 1
        if prediction == i :
          correct += 1
      
      scores[j][i].append(correct/total * 100)

print(scores)

100%|████████████████████████████████████████| 338M/338M [00:03<00:00, 116MiB/s]


tensor([[49406,   320,  1125,   539,   320,  3835, 49407,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [49406,   320,  1125,   539,   320,   811,  3699, 49407,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0, 

In [5]:
for j in range(5) :
  for i in range(5) :
    print(scores[j][i])
  print()

[99.41176470588235, 85.6, 100.0]
[100.0, 100.0, 100.0]
[100.0, 100.0, 100.0]
[100.0, 100.0, 100.0]
[98.828125, 97.86096256684492, 99.48186528497409]

[99.41176470588235, 94.39999999999999, 100.0]
[99.79591836734694, 99.75369458128078, 100.0]
[100.0, 100.0, 100.0]
[99.12536443148689, 100.0, 100.0]
[97.265625, 93.85026737967914, 98.96373056994818]

[99.41176470588235, 92.0, 100.0]
[100.0, 100.0, 100.0]
[100.0, 100.0, 100.0]
[100.0, 100.0, 100.0]
[98.046875, 89.83957219251337, 99.13644214162349]

[99.41176470588235, 94.93333333333334, 100.0]
[100.0, 99.75369458128078, 100.0]
[100.0, 100.0, 100.0]
[100.0, 100.0, 100.0]
[93.75, 86.63101604278076, 96.89119170984456]

[99.41176470588235, 93.33333333333333, 100.0]
[100.0, 100.0, 100.0]
[100.0, 100.0, 100.0]
[99.70845481049562, 100.0, 100.0]
[97.65625, 96.52406417112299, 99.3091537132988]



In [12]:
print(torch.mean(torch.tensor(scores[0])))
print(torch.mean(torch.tensor(scores[1])))
print(torch.mean(torch.tensor(scores[2])))
print(torch.mean(torch.tensor(scores[3])))
print(torch.mean(torch.tensor(scores[4])))

tensor(98.7455)
tensor(98.8378)
tensor(98.5623)
tensor(98.0914)
tensor(99.0629)


In [16]:
for j in range(5) :
  print(torch.mean(torch.tensor(scores[j])))
  for i in range(5) :
    print(torch.mean(torch.tensor(scores[j][i])))
  print()

s = [0,0,0,0,0]

for j in range(5) :
  for i in range(5) :
    s[i] += int(torch.mean(torch.tensor(scores[j][i])))

for i in range(5) :
  print(s[i]/5)


tensor(98.7455)
tensor(95.0039)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(98.7237)

tensor(98.8378)
tensor(97.9373)
tensor(99.8499)
tensor(100.)
tensor(99.7085)
tensor(96.6932)

tensor(98.5623)
tensor(97.1373)
tensor(100.)
tensor(100.)
tensor(100.)
tensor(95.6743)

tensor(98.0914)
tensor(98.1150)
tensor(99.9179)
tensor(100.)
tensor(100.)
tensor(92.4241)

tensor(99.0629)
tensor(97.5817)
tensor(100.)
tensor(100.)
tensor(99.9028)
tensor(97.8298)

96.8
99.6
100.0
99.6
95.6
